In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from datetime import datetime as dt

%matplotlib inline

In [2]:
raw_data = pd.read_csv("/Users/stephan/Data_Science/Metis/Bootcamp/Data_project1/data_2019.csv")

We found that the `EXITS` column contains trailing spaces, so we get rid of those

In [3]:
raw_data.columns = raw_data.columns.str.rstrip()

Adding a Datetime column

In [4]:
raw_data["DATE_TIME"] = pd.to_datetime(raw_data.DATE + " " + raw_data.TIME, 
                                            format="%m/%d/%Y %H:%M:%S")

The data is organized by turnstile, but does not have a direct unique identifier for each turnstile. To create one we add the column `ID`, which concatenates into a unique turnstile ID.

In [5]:
raw_data["ID"] = raw_data['C/A']+raw_data['UNIT']+raw_data['SCP']+raw_data['STATION']

The data contains duplicates, so we deal with those next

In [6]:
# Get rid of the duplicate entries
raw_data.sort_values(["ID", "DATE_TIME"], 
                          inplace=True, ascending=False)
raw_data.drop_duplicates(subset=["ID", "DATE_TIME"], inplace=True)

In our design phase we decided to give our recommendations to the client in the form of stations to canvas, so we will group our data by station. We also move towards daily entries and exits instead of per 4 hours.

The data given for `ENTRIES` and `EXITS` is cumulative, so aggregating by the first value for each day gives us the cumulative count at the start of each day. We then calculate the difference between days to get the daily numbers

In [7]:
data_by_station = raw_data.groupby(['STATION','ID','DATE'])['ENTRIES', 'EXITS'].first().diff()
data_by_station.dropna(subset=["ENTRIES","EXITS"], axis=0, inplace=True)
data_by_station.reset_index(inplace=True)

<ipython-input-7-b79019d558d4>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_by_station = raw_data.groupby(['STATION','ID','DATE'])['ENTRIES', 'EXITS'].first().diff()


First analysis of the resulting data shows some problems
* negative values, indicating turnstile tickers that count down instead of up
* Very large values that indicate a reset of a ticker

We take the absolute values for the negative ticker data and set a limit for the maximum reasonable value of people going through a turnstile. Our limit is based on a max of 1 person per second, or 86400 a day. Anything higher than this number, we replace it with the mean for the rest of the values for that turnstile.

In [8]:
def get_daily_counts_entries(row, max_counter):
    if row["ENTRIES"] < 0:
        # Maybe counter is reversed?
        row["ENTRIES"] = abs(row["ENTRIES"])
    if row["ENTRIES"] > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        row["ENTRIES"] = np.nan
    return row["ENTRIES"]

data_by_station["ENTRIES"] = data_by_station.apply(get_daily_counts_entries, axis=1, max_counter=86400)

In [9]:
def get_daily_counts_exits(row, max_counter):
    if row["EXIT"] < 0:
        # Maybe counter is reversed?
        row["EXIT"] = abs(row["EXIT"])
    if row["ENTRIES"] > max_counter:
        # Check it again to make sure we're not still giving a counter that's too big
        row["EXIT"] = np.nan
    return row["EXIT"]

data_by_station["EXITS"] = data_by_station.apply(get_daily_counts_entries, axis=1, max_counter=86400)

We replace the extremely large values from resets with the mean for that turnstile

In [10]:
data_by_station['ENTRIES'] = data_by_station['ENTRIES'].fillna(data_by_station.groupby('ID')['ENTRIES'].transform('mean'))
data_by_station['EXITS'] = data_by_station['EXITS'].fillna(data_by_station.groupby('ID')['EXITS'].transform('mean'))

For canvassing we are interested in total traffic, so we add up entries and exits

In [11]:
data_by_station["TRAFFIC"] = data_by_station.ENTRIES + data_by_station.EXITS

In [12]:
data_by_station.to_csv('data_2019_clean.csv')